In [84]:
import pickle

import pandas as pd
import numpy as np
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import *

from collections import Counter

In [35]:
# 使用networkx加载图
adj_dict = np.load('mydata/adj_list.npy', allow_pickle=True).item()
G = nx.from_dict_of_lists(adj_dict)

In [ ]:
# 特征
features = pd.read_csv('mydata/features.csv', header=None)
features = np.array(features)

In [38]:
# 标签
label = pd.read_csv('mydata/label.csv', header=None)
label = np.array(label)
label = label[0]

In [43]:
label

array([0, 0, 0, ..., 6, 6, 6], dtype=int64)

In [86]:
len(label)

69464

In [44]:
features = torch.tensor(features, dtype=torch.float)
label = torch.tensor(label, dtype=torch.long)

D:\anaconda\envs\lightgbm_catboost\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [46]:
# 将图转为pyg的data类型
data = from_networkx(G)
data.x = features
data.y = label
#创建完成，打印图
print(data)

Data(edge_index=[2, 2915867], num_nodes=69464, x=[69464, 78], y=[69464])


In [81]:
data.has_isolated_nodes()

True

In [85]:
g = to_networkx(data)

In [88]:
arr = []
for k, v in adj_dict.items():
    arr.append(len(v))
    

In [91]:
sorted(arr, reverse=True)

[1981,
 1981,
 1981,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1980,
 1944,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1943,
 1367,
 1356,
 1356,
 1347,
 1345,
 1344,
 1339,
 1339,
 1339,
 1333,
 1333,
 1333,
 1333,
 1322,
 1322,
 1302,
 1301,
 1301,
 1298,
 1298,
 1298,
 1298,
 1297,
 1297,
 1297,
 1297,
 1297,
 1297,
 1296,
 1296,
 1296,
 1296,
 1296,
 1296,
 1294,
 1293,
 1292,
 1292,
 1292,
 1292,
 1292,
 1291,
 1290,
 1290,
 1289,
 1288,
 1288,
 1288,
 1288,
 1287,
 1286,
 1286,
 1286,
 1286,
 1286,
 1286,
 1286,
 1285,
 1285,
 1285,
 1285,
 1285,
 1285,
 1285,
 1284,
 1284,
 1284,
 1284,
 1283,
 1283,
 1283,
 1283,
 1283,
 1283,
 1283,
 1282,
 1282,
 1282,
 1282,
 1282,
 1282,
 1282,
 1282,
 1282,
 1282,

In [93]:
arr = np.array(arr)

In [96]:
len(arr[arr==0])

735

In [53]:
num_class = 12
features_num = 78
hidden_size = 16

In [69]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv,ResGatedGraphConv,GATv2Conv,SAGEConv
import torch.nn.functional as F
import torch.nn as nn

In [106]:
max_graph = None

In [107]:
def graph_showing(data):
    '''
    args:
         data: torch_geometric.data.Data
    '''

    G = nx.Graph()
    edge_index = data['edge_index'].t()
    label = data.y
    #     print(edge_index)
    edge_index = np.array(edge_index.cpu())
    #     print(edge_index)

    G.add_edges_from(edge_index)
    print(nx.is_connected(G))
    print(nx.number_connected_components(G))
    nodes = 0
    g = None
    for c in nx.connected_components(G):
        # 绘制子图
        subgraph = G.subgraph(c)
        if nodes < len(subgraph.nodes):
            g = subgraph
            nodes = len(subgraph.nodes)
    print(nodes)
    print(g)
    a = nx.number_of_edges(g)
    print(a)
    global max_graph
    max_graph = g

In [108]:
graph_showing(data)

False
13782
11528

1207615


In [114]:
max_data = from_networkx(max_graph)

In [118]:
print(max_data.y)

None


In [122]:
torch.autograd.set_detect_anomaly(True)
class GAT_NET(torch.nn.Module):
    def __init__(self, features, hidden, classes):
        super(GAT_NET, self).__init__()
        self.gat1 = SAGEConv(features, hidden, normalize=True)
        self.gat2 = SAGEConv(hidden, classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = self.gat2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
    
device = torch.device('cpu')
model = GAT_NET(features_num, hidden_size, num_class).to(device)  # 定义GAT
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn = nn.CrossEntropyLoss().to(device)
model.train()
for epoch in range(2):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out, label)
    print(loss)
    loss.backward()
    optimizer.step()


tensor(nan, grad_fn=<NllLossBackward0>)


D:\anaconda\envs\lightgbm_catboost\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: Error detected in LogSoftmaxBackward0. Traceback of forward call that caused the error:
  File "D:\anaconda\envs\lightgbm_catboost\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda\envs\lightgbm_catboost\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda\envs\lightgbm_catboost\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\anaconda\envs\lightgbm_catboost\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\anaconda\envs\lightgbm_catboost\lib\site-packages\ipykernel\kernelapp.py", line 619, in start
    self.io_loop.start()
  File "D:\anaconda\envs\lightgbm_catboost\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "D:\anaconda\envs\lightgbm_cat

RuntimeError: Function 'LogSoftmaxBackward0' returned nan values in its 0th output.